# Miniproject 1
Fingermovements predicting

In [ ]:

# IMPORTS
import torch
from torch import Tensor, nn
from torch.autograd import Variable

# Our own code
import helpers as HL
import models as ML

%reload_ext autoreload
%autoreload 2

#### Importing data

In [ ]:
train_input, train_target, test_input, test_target = HL.import_data(flatten=True)

### Create new models if you want

### Create different models model and define loss criterion and optimizer

In [ ]:
input_dim = train_input.size()[1]
output_dim = 1
nb_hidden_layers = 2
hidden_width = 500
# initialize the model
densemodel_1 = ML.DenseNet(input_dim, output_dim, nb_hidden_layers=2)
linear_model = ML.Linear_regression_model(input_dim, output_dim)


### Choose model and define loss criterion and optimizer

In [ ]:

model = densemodel_1
# define loss criterion
criterion = nn.MSELoss()
# define optimizer function and input based on model
learning_rate = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training model

In [ ]:
epochs = 10000 #2000

for epoch in range(epochs):
    epoch += 1
    
    inputs = Variable(train_input)
    labels = Variable(train_target.float())
    
    # clear gradients
    optimizer.zero_grad()
    
    outputs = model.forward(inputs)
    loss = criterion(outputs, labels)
    loss.backward() # "backpropagation" only one layer
    optimizer.step() # updates parameters in model
    preds = outputs.round().abs().view(-1)
    #print(preds)
    #print(labels)
    if epoch % 1000 == 0:
        print("error percantage: ", (int(sum((preds != labels)))/ 316))
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))
    
    # linear model got 0.012 - 0.020 error with lr=0.0001, MSE with 100000 epochs
    # densemodel_1 got 0.005 - 0.009 error with lr=0.0001, MSE with 10000 epochs, 2 hidden layers